In [1]:
from __future__ import print_function, division
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import os
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
class CurrencyDataset(Dataset):
    def __init__(self,root_dir):
        self.dir = os.path.abspath(root_dir)
    def __getitem__(self,idx):
        data_path = os.path.join(self.dir,'Batches','Batch_' + str(idx) + '.npy')
        label_path = os.path.join(self.dir,'Labels','Label_' + str(idx) + '.npy')
        data = np.load(data_path)
        labels = np.load(label_path)
        return (data,labels)
    def __len__(self):
        return len(os.listdir(os.path.join(self.dir,'Batches')))

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,512,(1,1))
        self.conv2 = nn.Conv2d(512,1024,(2,1))
        self.conv3 = nn.Conv2d(1024,2048,(3,1))
        self.conv4 = nn.Conv2d(2048,4096,(8,1))
        self.conv5 = nn.Conv2d(4096,2048,(3,1))
        self.conv6 = nn.Conv2d(2048,1024,(3,1))
        self.conv7 = nn.Conv2d(1024,1024,(3,1))
        self.conv8 = nn.Conv2d(1024,10000,(1,1))
        self.conv9 = nn.Conv2d(10000,10000,(1,1))
        self.conv10 = nn.Conv2d(10000,1,(1,1))
        self.activation = F.relu
        self.pool = nn.AvgPool2d((3,1))
        self.norm_512 = nn.BatchNorm2d(512)
        self.norm_1024 = nn.BatchNorm2d(1024)
        self.norm_2048 = nn.BatchNorm2d(2048)
        self.norm_4096 = nn.BatchNorm2d(4096)
        self.norm_10000 = nn.BatchNorm2d(10000)
#         self.conv_test = nn.Conv2d(5,128,(48,3))
#         self.conv_test2 = nn.Conv2d(128,5,(3,1))
        self.fc1 = nn.Linear(5, 5)
        self.dp1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(3000, 3500)
        self.fc3 = nn.Linear(3500, 5)

    def forward(self, x):
     #   x = self.dp1(x)
#         print(x.shape)
        x = self.activation(self.conv1(x))
#         print(x.shape)
        x = self.activation(self.conv2(x))
#         print(x.shape)
        x = self.activation(self.pool(x))
#         print(x.shape)
        x = self.activation(self.norm_2048(self.conv3(x)))
#         print(x.shape)
        x = self.activation(self.norm_4096(self.conv4(x)))
#         print(x.shape)
        x = self.activation(self.norm_2048(self.conv5(x)))
#         print(x.shape)
        x = self.activation(self.norm_1024(self.conv6(x)))
#         print(x.shape)
        x = self.activation(self.norm_1024(self.conv7(x)))
#         print(x.shape)
        x = self.activation(self.norm_10000(self.conv8(x)))
        x = self.activation(self.norm_10000(self.conv9(x)))
        x = self.conv10(x)
#         print(x.shape)
#         x = F.relu(self.conv_test(x))
#         x = self.conv_test2(x)
        
#         print(x.shape)
#         print(x.shape)
#         x = x.view(-1,5)
#         print(x.shape)
#         x = x.view(-1,5)
#         print(x.shape)       
#         x = F.relu(self.fc1(x))
#         print(x.shape)
# #         x = self.dp1(x)
#         x = F.relu(self.fc2(x))
# #         x = self.dp1(x)
#         x = self.fc3(x)
        x = x.view(-1,5)
#         x = self.dp1(x)
#         print(x.shape)
        
    
        x = F.softmax(x,dim=1)
        return x
def weight_reset(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)

In [3]:
torch.manual_seed(991)
torch.set_default_tensor_type('torch.cuda.FloatTensor')
data = CurrencyDataset('./Processed')
device = 'cuda:1'
currency_count = 5
net = Net()
# net = nn.DataParallel(net,device_ids=[0])
# net.to(device)
# net = net.half()
net = net.cuda()
net.train()
# dataloader = DataLoader(data,batch_size=256,pin_memory=True, sampler=SubsetRandomSampler(list(range(2048))))
dataloader = DataLoader(data,batch_size=128,pin_memory=True,shuffle=True)

In [4]:
optimizer = optim.Adam(net.parameters(), lr=0.0002)

In [ ]:
net.apply(weight_reset)
# test_X, test_y = iter(dataloader).next()
# print(test_X)
# print(test_y)
print_every = 16
begin = time.time()
for epoch in range(50):
    epoch_loss = 0.0
    running_loss = 0.0
    j = 0
    for i,batch in enumerate(dataloader, 0):
        j = i
        optimizer.zero_grad()
        d,y = batch
        d = d.float()
        y = y.float()
#         d = test_X
#         y = test_y
        d = d.cuda()
        x = net(d)
        y = (y.reshape([-1,5])).cuda()
#         print(x.shape)
#         print(y.shape)
        z = torch.mul(x,y)
        z = z.sum(dim=1)
        loss = z.sum()
        loss /= z.shape[0]
        loss = loss ** -1
        epoch_loss += loss.item() * 10000
        running_loss += loss.item() * 10000
        
        if i % print_every == print_every - 1:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / print_every))
            print('Time elapsed: {} minutes'.format((time.time() - begin)/60))
            running_loss = 0.0

        loss.backward()
        optimizer.step()
    print('Epoch {} loss: {}'.format(epoch+1, epoch_loss/(j+1)))
end = time.time()
print("FINAL TIME: {}".format(end-begin))

[1,    16] loss: 10000.008
Time elapsed: 0.18204181989034016 minutes
[1,    32] loss: 10000.005
Time elapsed: 0.33903433084487916 minutes
[1,    48] loss: 10000.003
Time elapsed: 0.49641729990641276 minutes
[1,    64] loss: 10000.002
Time elapsed: 0.6545615315437316 minutes
[1,    80] loss: 10000.002
Time elapsed: 0.8123891671498616 minutes
[1,    96] loss: 9999.998
Time elapsed: 0.9702796697616577 minutes
[1,   112] loss: 10000.006
Time elapsed: 1.1284828702608745 minutes
[1,   128] loss: 10000.004
Time elapsed: 1.2866618673006693 minutes
[1,   144] loss: 10000.003
Time elapsed: 1.445622718334198 minutes
[1,   160] loss: 10000.000
Time elapsed: 1.604663574695587 minutes
[1,   176] loss: 10000.008
Time elapsed: 1.7637044151624044 minutes
[1,   192] loss: 10000.000
Time elapsed: 1.9229563355445862 minutes
[1,   208] loss: 10000.031
Time elapsed: 2.0819271047910055 minutes
[1,   224] loss: 10000.002
Time elapsed: 2.2412876884142556 minutes
[1,   240] loss: 10000.005
Time elapsed: 2.40067

[1,  1968] loss: 9999.987
Time elapsed: 19.787444670995075 minutes
[1,  1984] loss: 10000.005
Time elapsed: 19.949125941594442 minutes
[1,  2000] loss: 9999.993
Time elapsed: 20.110376075903574 minutes
[1,  2016] loss: 10000.013
Time elapsed: 20.27212672630946 minutes
[1,  2032] loss: 9999.978
Time elapsed: 20.43371944030126 minutes
[1,  2048] loss: 10000.022
Time elapsed: 20.595164855321247 minutes
[1,  2064] loss: 9999.991
Time elapsed: 20.75671164194743 minutes
[1,  2080] loss: 10000.018
Time elapsed: 20.918401583035788 minutes
[1,  2096] loss: 9999.987
Time elapsed: 21.080150079727172 minutes
[1,  2112] loss: 9999.952
Time elapsed: 21.24203755458196 minutes
[1,  2128] loss: 10000.029
Time elapsed: 21.403904044628142 minutes
[1,  2144] loss: 10000.012
Time elapsed: 21.56545407772064 minutes
[1,  2160] loss: 10000.029
Time elapsed: 21.727362648646036 minutes
[1,  2176] loss: 10000.032
Time elapsed: 21.889136175314587 minutes
[1,  2192] loss: 10000.005
Time elapsed: 22.050681885083517

[1,  3920] loss: 10000.001
Time elapsed: 39.518949437141416 minutes
[1,  3936] loss: 9999.990
Time elapsed: 39.68101341724396 minutes
[1,  3952] loss: 9999.992
Time elapsed: 39.843007465203605 minutes
[1,  3968] loss: 10000.011
Time elapsed: 40.00508110920588 minutes
[1,  3984] loss: 10000.003
Time elapsed: 40.166908045609794 minutes
[1,  4000] loss: 10000.003
Time elapsed: 40.328872354825336 minutes
[1,  4016] loss: 10000.000
Time elapsed: 40.490989867846174 minutes
[1,  4032] loss: 10000.003
Time elapsed: 40.65285443464915 minutes
[1,  4048] loss: 10000.015
Time elapsed: 40.81511875391006 minutes
[1,  4064] loss: 9999.999
Time elapsed: 40.97679911454519 minutes
[1,  4080] loss: 10000.010
Time elapsed: 41.13870042562485 minutes
Epoch 1 loss: 9999.409064212266
[2,    16] loss: 9999.986
Time elapsed: 41.31166133085887 minutes
[2,    32] loss: 9999.992
Time elapsed: 41.47341215610504 minutes
[2,    48] loss: 9999.996
Time elapsed: 41.63525322675705 minutes
[2,    64] loss: 10000.006
Time

[2,  1808] loss: 9999.998
Time elapsed: 59.44901170333227 minutes
[2,  1824] loss: 10000.015
Time elapsed: 59.610789628823596 minutes
[2,  1840] loss: 10000.008
Time elapsed: 59.772933117548625 minutes
[2,  1856] loss: 10000.004
Time elapsed: 59.93523124059041 minutes
[2,  1872] loss: 10000.002
Time elapsed: 60.09747730096181 minutes
[2,  1888] loss: 10000.005
Time elapsed: 60.2599884112676 minutes
[2,  1904] loss: 10000.015
Time elapsed: 60.42229337692261 minutes
[2,  1920] loss: 10000.007
Time elapsed: 60.584373319149016 minutes
[2,  1936] loss: 10000.006
Time elapsed: 60.74657473564148 minutes
[2,  1952] loss: 10000.011
Time elapsed: 60.90870203574499 minutes
[2,  1968] loss: 10000.007
Time elapsed: 61.0708212574323 minutes
[2,  1984] loss: 9999.998
Time elapsed: 61.2330085237821 minutes
[2,  2000] loss: 9999.996
Time elapsed: 61.395069273312885 minutes
[2,  2016] loss: 9999.999
Time elapsed: 61.557112809022264 minutes
[2,  2032] loss: 10000.013
Time elapsed: 61.71906017065048 minut

[2,  3792] loss: 10000.005
Time elapsed: 79.55142724911371 minutes
[2,  3808] loss: 9999.996
Time elapsed: 79.71362431049347 minutes
[2,  3824] loss: 9999.982
Time elapsed: 79.87551327546437 minutes
[2,  3840] loss: 10000.006
Time elapsed: 80.03737380504609 minutes
[2,  3856] loss: 10000.005
Time elapsed: 80.19980054696401 minutes
[2,  3872] loss: 10000.011
Time elapsed: 80.36193129221598 minutes
[2,  3888] loss: 10000.007
Time elapsed: 80.52389818827311 minutes
[2,  3904] loss: 9999.990
Time elapsed: 80.68630297978719 minutes
[2,  3920] loss: 10000.005
Time elapsed: 80.84836265246074 minutes
[2,  3936] loss: 10000.016
Time elapsed: 81.0104400396347 minutes
[2,  3952] loss: 9999.982
Time elapsed: 81.17255436579386 minutes
[2,  3968] loss: 9999.992
Time elapsed: 81.33420145908991 minutes
[2,  3984] loss: 10000.003
Time elapsed: 81.49626327355703 minutes
[2,  4000] loss: 10000.010
Time elapsed: 81.65849877595902 minutes
[2,  4016] loss: 9999.985
Time elapsed: 81.8209310332934 minutes
[2,

[3,  1680] loss: 9999.999
Time elapsed: 99.49964774052302 minutes
[3,  1696] loss: 9999.991
Time elapsed: 99.66166181564331 minutes
[3,  1712] loss: 10000.014
Time elapsed: 99.82387498219808 minutes
[3,  1728] loss: 9999.991
Time elapsed: 99.98603855371475 minutes
[3,  1744] loss: 10000.005
Time elapsed: 100.14815067052841 minutes
[3,  1760] loss: 10000.014
Time elapsed: 100.310285226504 minutes
[3,  1776] loss: 9999.991
Time elapsed: 100.47271596988043 minutes
[3,  1792] loss: 9999.990
Time elapsed: 100.63481457630793 minutes
[3,  1808] loss: 10000.019
Time elapsed: 100.79711239337921 minutes
[3,  1824] loss: 10000.016
Time elapsed: 100.95910101334253 minutes
[3,  1840] loss: 9999.993
Time elapsed: 101.12110633850098 minutes
[3,  1856] loss: 10000.002
Time elapsed: 101.28318619330724 minutes
[3,  1872] loss: 9999.999
Time elapsed: 101.4448782881101 minutes
[3,  1888] loss: 10000.011
Time elapsed: 101.60709672371546 minutes
[3,  1904] loss: 10000.010
Time elapsed: 101.76925867795944 mi

[3,  3632] loss: 10000.033
Time elapsed: 119.27332918643951 minutes
[3,  3648] loss: 9999.995
Time elapsed: 119.4353577097257 minutes
[3,  3664] loss: 10000.003
Time elapsed: 119.5978302081426 minutes
[3,  3680] loss: 9999.997
Time elapsed: 119.75999925136566 minutes
[3,  3696] loss: 9999.996
Time elapsed: 119.92193889220556 minutes
[3,  3712] loss: 10000.012
Time elapsed: 120.08429261048634 minutes
[3,  3728] loss: 9999.993
Time elapsed: 120.24625479380289 minutes
[3,  3744] loss: 10000.008
Time elapsed: 120.40853190024694 minutes
[3,  3760] loss: 10000.015
Time elapsed: 120.57090696891149 minutes
[3,  3776] loss: 9999.993
Time elapsed: 120.7330883105596 minutes
[3,  3792] loss: 10000.007
Time elapsed: 120.89518601894379 minutes
[3,  3808] loss: 10000.020
Time elapsed: 121.05708281993866 minutes
[3,  3824] loss: 10000.003
Time elapsed: 121.21890161832174 minutes
[3,  3840] loss: 9999.995
Time elapsed: 121.38068109750748 minutes
[3,  3856] loss: 10000.007
Time elapsed: 121.543014140923

[4,  1504] loss: 10000.013
Time elapsed: 139.05833337306976 minutes
[4,  1520] loss: 9999.997
Time elapsed: 139.22058802048366 minutes
[4,  1536] loss: 9999.998
Time elapsed: 139.38273901542027 minutes
[4,  1552] loss: 10000.010
Time elapsed: 139.5451429128647 minutes
[4,  1568] loss: 10000.014
Time elapsed: 139.70724493265152 minutes
[4,  1584] loss: 9999.995
Time elapsed: 139.86978404124577 minutes
[4,  1600] loss: 9999.988
Time elapsed: 140.03205958604812 minutes
[4,  1616] loss: 9999.990
Time elapsed: 140.19406027793883 minutes
[4,  1632] loss: 9999.998
Time elapsed: 140.35638063748678 minutes
[4,  1648] loss: 10000.032
Time elapsed: 140.5185476700465 minutes
[4,  1664] loss: 10000.020
Time elapsed: 140.68082880973816 minutes
[4,  1680] loss: 9999.992
Time elapsed: 140.84319881598154 minutes
[4,  1696] loss: 10000.020
Time elapsed: 141.00517156918843 minutes
[4,  1712] loss: 10000.009
Time elapsed: 141.16732871135076 minutes
[4,  1728] loss: 10000.002
Time elapsed: 141.329626568158

In [ ]:
# net.eval()
# for i,batch in enumerate(dataloader):
#     d,y = batch
#     print(y)
#     print(net(d.cuda()))
#     break